In [8]:
# Challenge 1 - 2025/05/27
# - Make a chain that takes the name of a movie and replies with information about the movie such as the director, the main cast, the budget, the box office revenue, the genre of the movie and a small synopsis.
# - The LLM should always reply using the same format, to accomplish this you must give examples to the LLM of your desired output.
# - To provide examples use FewShotPromptTemplate or FewShotChatMessagePromptTemplate.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about Harry potter?",
        "answer": """
        title: Harry Potter and the Sorcerer's Stone,
        director: Chris Columbus,
        main_cast: 
            Daniel Radcliffe as Harry Potter,
            Emma Watson as Hermione Granger,
            Rupert Grint as Ron Weasley,
            Richard Harris as Albus Dumbledore,
            Alan Rickman as Severus Snape
        ,
        budget: $125 million,
        box_office: $974 million worldwide,
        genre: Fantasy, Adventure, Family,
        synopsis: Harry Potter, an orphaned boy living with his unkind relatives, discovers on his 11th birthday that he is a wizard. He attends Hogwarts School of Witchcraft and Wizardry where he makes friends, uncovers mysteries about his past, and learns about a dark wizard named Lord Voldemort who once tried to kill him.
        """,
    },
    {
        "question": "What do you know about Avengers3?",
        "answer": """
        title: Avengers: Infinity War,
        director: Anthony and Joe Russo,
        main_cast: 
            Robert Downey Jr. as Tony Stark / Iron Man,
            Chris Hemsworth as Thor,
            Chris Evans as Steve Rogers / Captain America,
            Mark Ruffalo as Bruce Banner / Hulk,
            Scarlett Johansson as Natasha Romanoff / Black Widow,
            Josh Brolin as Thanos,
            Tom Holland as Peter Parker / Spider-Man
        ,
        budget: $325–400 million,
        box_office: $2.048 billion worldwide,
        genre: Action, Adventure, Sci-Fi,
        synopsis: The Avengers and their allies must be willing to sacrifice everything in an attempt to defeat the powerful Thanos before his blitz of devastation and ruin puts an end to half of all life in the universe.
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "The lord of the rings"})

AI: 
        title: The Lord of the Rings,
        director: Peter Jackson,
        main_cast: 
            Elijah Wood as Frodo Baggins,
            Ian McKellen as Gandalf the Grey,
            Viggo Mortensen as Aragorn,
            Orlando Bloom as Legolas,
            Sean Astin as Samwise Gamgee,
            Liv Tyler as Arwen,
            Cate Blanchett as Galadriel
        ,
        budget: $281 million for the entire trilogy,
        box_office: $2.981 billion worldwide for the entire trilogy,
        genre: Fantasy, Adventure,
        synopsis: The Lord of the Rings is a epic fantasy trilogy based on the novels by J.R.R. Tolkien. It follows the journey of a young hobbit named Frodo Baggins as he sets out to destroy the One Ring and defeat the Dark Lord Sauron. Along the way, he is joined by a fellowship of diverse characters who help him in his quest to save Middle-earth from darkness.

AIMessageChunk(content='AI: \n        title: The Lord of the Rings,\n        director: Peter Jackson,\n        main_cast: \n            Elijah Wood as Frodo Baggins,\n            Ian McKellen as Gandalf the Grey,\n            Viggo Mortensen as Aragorn,\n            Orlando Bloom as Legolas,\n            Sean Astin as Samwise Gamgee,\n            Liv Tyler as Arwen,\n            Cate Blanchett as Galadriel\n        ,\n        budget: $281 million for the entire trilogy,\n        box_office: $2.981 billion worldwide for the entire trilogy,\n        genre: Fantasy, Adventure,\n        synopsis: The Lord of the Rings is a epic fantasy trilogy based on the novels by J.R.R. Tolkien. It follows the journey of a young hobbit named Frodo Baggins as he sets out to destroy the One Ring and defeat the Dark Lord Sauron. Along the way, he is joined by a fellowship of diverse characters who help him in his quest to save Middle-earth from darkness.')